In [1]:
import spacy
import json
import random
import os
from spacy.training import Example


#### merge all jsonl files and put in 1

In [2]:
# # Directory containing the JSONL files
# source_folder = os.path.expanduser('~/Work/self/coffee/coffee_database/ner/data')

# # Output file in the current working directory
# output_file = 'data/merged_data.jsonl'

# # Get all the jsonl files in the folder
# jsonl_files = [f for f in os.listdir(source_folder) if f.endswith('.jsonl')]

# # Merge all jsonl files into one
# with open(output_file, 'w') as outfile:
#     for jsonl_file in jsonl_files:
#         with open(os.path.join(source_folder, jsonl_file), 'r') as infile:
#             for line in infile:
#                 outfile.write(line)

# print(f"All JSONL files have been merged into {output_file}")


#### converting in spacy acceptable training data

In [3]:
# reading the data in doccano jsonl output format
with open('data/merged_data.jsonl', 'r') as f:
    lines = list(f)

training_data: list = []

for line in lines:
    row = json.loads(line)
    if row['label']:
        training_data.append(  [ row["text"], { "entities": row["label"] } ] )

print(len(training_data))

random.shuffle(training_data)

183


In [4]:
training_data[30:31]

[['"tastes like-cacao nibs , dark chocolate and long finsh | roasting profile-espresso ( dark roast ) | varietal-catura , sln-795 | altitude-1260masl | process-washed arabica | harvest year-2023 | location-chikmagalur welcome to the dark side of coffee ! well in all honesty we dont judge and believe to each their own . frankly any coffee roasted this dark shall taste the same since the sugars get caramelized to an extent that you shall find smoky-toasty-roasty notes . people love it and we love you legends"',
  {'entities': [[13, 23, 'TASTING NOTES'],
    [26, 40, 'TASTING NOTES'],
    [45, 49, 'AFTERTASTE'],
    [86, 90, 'ROAST LEVEL'],
    [110, 116, 'VARIETAL'],
    [119, 126, 'VARIETAL'],
    [138, 146, 'ELEVATION'],
    [157, 163, 'PROCESSING'],
    [164, 171, 'COFFEE TYPE'],
    [203, 214, 'LOCATION']]}]]

In [5]:
# Add labels to the NER training_data
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        print(ent[2])

ESTATE
TASTING NOTES
TASTING NOTES
TASTING NOTES
PROCESSING
VARIETAL
LOCATION
ESTATE
ESTATE
TASTING NOTES
ROAST LEVEL
BODY
BODY
AFTERTASTE
AFTERTASTE
TASTING NOTES
ROAST LEVEL
ELEVATION
PROCESSING
COFFEE TYPE
FERMENTATION TIME
ROAST LEVEL
COFFEE TYPE
ESTATE
ESTATE
COFFEE TYPE
ESTATE
TASTING NOTES
TASTING NOTES
TEXTURE
AROMA
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
VARIETAL
ROAST LEVEL
ESTATE
ELEVATION
TASTING NOTES
BODY
TASTING NOTES
ROAST LEVEL
ROAST LEVEL
ELEVATION
PROCESSING
COFFEE TYPE
ACIDITY
BODY
PROCESSING
PROCESSING
TEXTURE
TASTING NOTES
ESTATE
ROAST LEVEL
COFFEE_PROPERTIES
ESTATE
ESTATE
LOCATION
LOCATION
PROCESSING
ELEVATION
TASTING NOTES
TASTING NOTES
TASTING NOTES
ESTATE
LOCATION
LOCATION
PROCESSING
FERMENTATION TIME
FERMENTATION TIME
PROCESSING
ACIDITY
BODY
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
ESTATE
ELEVATION
VARIETAL
VARIETAL
TASTING NOTES
TASTING NOTES
TASTING NOTES
TASTING NOTES
ACIDITY
AFTERTAS

In [6]:
def train_spacy(TRAIN_DATA, iterations):

    # Load a blank spaCy model or a pre-existing one
    # nlp = spacy.blank("en")  # or `spacy.load("en_core_web_sm")` if you want to fine-tune
    nlp = spacy.load("en_core_web_lg")

    # Add NER to the pipeline if it doesn't exist
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")
    # Add labels to the NER training_data

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Disable other pipeline components during training
    pipe_exceptions = ["ner"]
    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

    # Training the NER model
    with nlp.disable_pipes(*unaffected_pipes):  # Only train NER
        optimizer = nlp.begin_training()
        for iteration in range(iterations):  # Adjust the number of iterations as needed
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.2, losses=losses, sgd=optimizer)
            print(f"Iteration {iteration}: Losses {losses}")

    return nlp

In [7]:
trained = train_spacy(training_data,50)

/Users/snehil/.dev_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/snehil/.dev_env/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text ""elevation : 3200-4250 ft . varietal : sl6 , sln9 ..." with entities "[[13, 24, 'ELEVATION'], [39, 42, 'VARIETAL'], [45,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/snehil/.dev_env/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text ""tastes like-caramel , hint of clove and white cho..." with entities "[[13, 20, 'TASTING NOTES'], [31, 36, 'TASTING NOTE...". Use `spacy.t

Iteration 0: Losses {'ner': 5195.933288856188}
Iteration 1: Losses {'ner': 3012.2163542282005}
Iteration 2: Losses {'ner': 2449.1141953915576}
Iteration 3: Losses {'ner': 2190.715392788494}
Iteration 4: Losses {'ner': 1979.0145080035468}
Iteration 5: Losses {'ner': 1971.3751349280426}
Iteration 6: Losses {'ner': 1849.5372145228225}
Iteration 7: Losses {'ner': 1560.410607716967}
Iteration 8: Losses {'ner': 1563.0984423608002}
Iteration 9: Losses {'ner': 1495.1149514192168}
Iteration 10: Losses {'ner': 1445.6445743653212}
Iteration 11: Losses {'ner': 1392.2206433981662}
Iteration 12: Losses {'ner': 1309.988589546034}
Iteration 13: Losses {'ner': 1299.4895238898391}
Iteration 14: Losses {'ner': 1297.64699583979}
Iteration 15: Losses {'ner': 1306.919885797311}
Iteration 16: Losses {'ner': 1444.8097997638004}
Iteration 17: Losses {'ner': 1218.9559536917984}
Iteration 18: Losses {'ner': 1165.1020258505168}
Iteration 19: Losses {'ner': 1117.7560821851728}
Iteration 20: Losses {'ner': 1130.673

In [8]:
sample_text="""


Heisenberg is more than just a coffee; since its explosive debut in 2020, this coffee has become an addiction. Just like Walter White's unforgettable meth, Heisenberg will keep you coming back for more, creating a bond that's as strong as the cravings it ignites. 

Heisenberg is renowned for his meticulous methods and ensures this coffee will blow you away. With his vast expertise and dedication, Mr. Prakashan Balaraman from Riverdale Estate has meticulously harvested and processed the beans to perfection. To bring out the best in every batch, we introduce Jesse Pinkman, aka trader Mr. Baninder Kochar, whose skill in profile roasting adds a unique touch to our coffee. Together, they ensure every cup you brew will be an exceptional experience that exceeds all expectations.

This coffee continues our love for Natural processed coffees because of its complexity in both flavor and its aromatic notes. The ripened cherries are hand-picked and triple-washed to remove all the floaters from the cherries. They are then processed in purpose-built tanks where CO2 is introduced to flush out the oxygen from the tanks to ferment the cherries for longer. The second stage of fermentation is introducing yeast along with various fruits such as Pineapple, Peach, Red Fruit, Watermelon, and grapes for 14 days when the fermentation happens slowly. The yeast thrives in the low oxygen environment and helps develop complex fruit notes. Once the fermentation is complete, cherries are slowly dried on raised beds to remove excess water and improve air circulation for even drying.

This process is complex and delicate. It took 4 years of research with the wine industry for the Heisenberg of the Indian Coffee Industry to get the right level of flavor. 

With the aroma of figs,  this coffee is a fruit party with tasting notes of Fig, Cranberry, Roasted Pineapple, and Port Wine. Experience the special formula that Heisenberg has implemented this time.

Facts:

Origin: Riverdale Estate
Altitude: 1450 MASL
Varietal: SLN-9
Process: Fruit Infused Carbonic Maceration

Feels:

Tasting Notes: Fig, Cranberry, Roasted Pineapple, Port Wine
Acidity: Malic
Body: Heavy
Aftertaste: Very High

Roast Rite Color (Whole Bean): 74
Roast Rite Color (Ground): 86
Roast Level: Light

"""
docs = trained(sample_text.lower())
for ent in docs.ents:
    print(ent.text,ent.label_)

heisenberg NAME
heisenberg NAME
heisenberg NAME
mr. prakashan balaraman FARMER
riverdale ESTATE
natural processed PROCESSING
red fruit TASTING NOTES
heisenberg NAME
fig TASTING NOTES
cranberry TASTING NOTES
roasted pineapple TASTING NOTES
port wine TASTING NOTES
heisenberg NAME
riverdale ESTATE
1450 masl ELEVATION
sln-9 VARIETAL
fruit infused carbonic maceration PROCESSING
fig TASTING NOTES
cranberry TASTING NOTES
roasted pineapple TASTING NOTES
port wine TASTING NOTES
malic ACIDITY
light ROAST LEVEL


In [9]:
# Save the trained model
trained.to_disk("model/coffee_ner_en_lg_1")